In [5]:
import math
from turtle import shape
from unittest import result
import pandas as pd
from re import M
import numpy as np
from sympy import *
import matplotlib.pyplot as plt


In [6]:
def equation(c1, Pw, MU, test):
    inv1 = np.linalg.inv(c1)
    tra = np.transpose(np.array(test - MU))
    p1 = np.dot(tra,inv1)
    p2 = np.dot(p1,np.array(test - MU))
    equation = (-0.5*p2) - (math.log(2*np.pi)) + math.log(Pw) 
    return equation

def meanval(data):
    MU = []
    for i in range(shape(data)[0]):
        s1 = 0.0
        for j in range(shape(data)[1]):
            s1 = s1 + data[i][j]
            
        MU1 = s1/shape(data)[1]
        MU.append(MU1)
    MU = list(MU)    
    return MU

In [7]:
data = pd.read_csv("ffv.csv")
data = data.drop(data.columns[0], axis=1)
data = data.drop(data.columns[0], axis=1)

# req data in dataframe format
male = data[10:400]
female = data[410:800]

testmale = data[0:10]
testfemale = data[400:410]

male = male.to_numpy()
female = female.to_numpy()

testmale = testmale.to_numpy()
testfemale = testfemale.to_numpy()

traindata = np.concatenate((male, female))

In [21]:
mu_m = []    
mu_f = []    

for i in range(128):
    mm = 0
    mf = 0
    for j in range(len(male)):
        mm = mm + male[j][i]
        mf = mf + female[j][i]
    mm = mm/390
    mf = mf/390
    mu_m.append(mm)
    mu_f.append(mf)


Zm = []
Zf = []

for i in range(len(male)):
    x = np.subtract(male[i],mu_m)
    y = np.subtract(female[i],mu_f)
    Zm.append(x)
    Zf.append(y)
    
# print(len(Zm))
Zmt = np.transpose(Zm)
Zft = np.transpose(Zf)

# print(len(Zmt))
COV_m = np.dot(Zmt, Zm) 
COV_f = np.dot(Zft, Zf) 

S = COV_m + COV_f
Sinv = np.linalg.inv(S) 
print("-----------------")
print(shape(S))
print("-----------------")
print(shape(Sinv))

MU = np.subtract(mu_m, mu_f)
MUt = np.transpose(MU)
Sb = []
print(shape(MU))
print(shape(MUt))
for t in range(len(MU)):
    prod = []
    for t_ in range(len(MUt)):
        p = MU[t] * MUt[t_]
        prod.append(p)    
    Sb.append(prod)

print("***********")
Sb = np.array(Sb)
print(shape(Sb))
print(Sb)
print("***********")

SW_Sb = np.dot(Sinv, Sb)
print(shape(SW_Sb))
# w = -np.sort(-w)
print(SW_Sb)

w_, v_ = np.linalg.eig(SW_Sb)
w = w_ 
w_ = -np.sort(-w_) 
print(v_)

p_max = np.where(w == w_[0])
print(str(w_[0]) +"  " + str(p_max))
W = v_[: ,p_max[0][0]]
print(W)

d_dash = 1
colval = []
for j in range(d_dash):
    p = np.where(w == w_[j])
    colval1 = np.dot(traindata, W)
    colval.append(colval1)

    Pw1 = 0.5
    Pw2 = 0.5
colvaltm = []
colvaltf = []

for j in range(d_dash):
    p = np.where(w == w_[j])
    colval1 = np.dot(testmale, W)
    colval2 = np.dot(testfemale, W)
    colvaltm.append(colval1)
    colvaltf.append(colval2)

    Pw1 = 0.5
    Pw2 = 0.5

print("**********")
print(colvaltm)
print(colvaltf)
print("**********")

colval = np.array(colval)

colval = np.transpose(colval)

print(shape(colval))

df = pd.DataFrame(colval)
dfm = pd.DataFrame(colvaltm)
dff = pd.DataFrame(colvaltf)


print(df.head())

df_male = df[0:390]
df_female = df[390:]

dfm = dfm.to_numpy()
dff = dff.to_numpy()

mc = df_male.cov()
fc = df_female.cov()

mc = mc.to_numpy()
fc = fc.to_numpy()

invcovmc = np.linalg.inv(mc)
invcovfc = np.linalg.inv(fc)

MU1 = meanval(np.transpose(df_male.to_numpy()))
MU2 = meanval(np.transpose(df_female.to_numpy()))
print(MU1)
print(shape(np.array(MU1)))
print(shape(np.transpose(dfm)))
print(shape(np.transpose(dff)))

dfm = np.transpose(dfm)
dff = np.transpose(dff)

m1=0
m2=0

print("Testing Male vectors")
for i in range(0,10):
    score1 = equation(mc,Pw1,MU1,dfm[i])
    score2 = equation(fc,Pw2,MU2,dfm[i])
    if(score2>score1):
        print("Female")
        m1 = m1 + 1
    else:
       print("Male")
print("Testing Female Vectors")
for i in range(0,10):
    score1 = equation(mc,Pw1,MU1,dff[i])
    score2 = equation(fc,Pw2,MU2,dff[i])
    if(score1>score2):
        print("Male")
        m2 = m2 + 1
    else:
        print("Female")
accuracy1 = (1 - m1/10)*100
accuracy2 = (1 - m2/10)*100
print("The Accuracy for Male: ",accuracy1,"%")
print("The Accuracy for Female: ",accuracy2,"%")
print("The Total Accuracy: ",(accuracy1+accuracy2)/2,"%")
print("CONFUSION MATRIX:")
CF = []
CF.append([(10-m1), m1])
CF.append([m2, (10-m2)])
print(str(CF[0][0])+" "+str(CF[0][1])+"\n"+str(CF[1][0])+" "+str(CF[1][1]))

-----------------
(128, 128)
-----------------
(128, 128)
(128,)
(128,)
***********
(128, 128)
[[ 7.39708162e-04  4.46063882e-04 -1.03471597e-03 ...  1.82479221e-04
   4.35584942e-04  3.67627347e-04]
 [ 4.46063882e-04  2.68988497e-04 -6.23961513e-04 ...  1.10039869e-04
   2.62669415e-04  2.21689161e-04]
 [-1.03471597e-03 -6.23961513e-04  1.44737776e-03 ... -2.55254942e-04
  -6.09303398e-04 -5.14243194e-04]
 ...
 [ 1.82479221e-04  1.10039869e-04 -2.55254942e-04 ...  4.50159504e-05
   1.07454811e-04  9.06902954e-05]
 [ 4.35584942e-04  2.62669415e-04 -6.09303398e-04 ...  1.07454811e-04
   2.56498780e-04  2.16481235e-04]
 [ 3.67627347e-04  2.21689161e-04 -5.14243194e-04 ...  9.06902954e-05
   2.16481235e-04  1.82707009e-04]]
***********
(128, 128)
[[-102.31846921  -61.70078407  143.12476163 ...  -25.24102812
   -60.25130819  -50.85122663]
 [-154.83369129  -93.36887297  216.58392017 ...  -38.19605185
   -91.17544978  -76.95075174]
 [  72.62042707   43.79206728 -101.58265074 ...   17.9147934

C:\Users\dhyut\anaconda3\lib\site-packages\pandas\core\internals\managers.py:1464: ComplexWarning: Casting complex values to real discards the imaginary part
  arr = arr.astype(dtype, copy=False)  # type: ignore[arg-type]


(128,)
(128,)


AttributeError: 'numpy.ndarray' object has no attribute 'append'